In [88]:
import os
import pandas as pd
import numpy as np
import re
import datetime
import pickle
import json
import time
from nltk.tokenize import sent_tokenize
import nltk
from collections import Counter

from nltk.stem import WordNetLemmatizer,PorterStemmer
  
lemmatizer = WordNetLemmatizer()
ps = PorterStemmer()

os.chdir('..')
print(os.getcwd())
from utils import get_fulltext,get_fname,fulltext_exists
os.chdir('./data_processing/')

/Users/yiweiluo/scientific-debates


In [2]:
df = pd.read_pickle('../data_scraping/dedup_combined_df.pkl')
df.head(10)

,url,title,date,domain,stance,topic,is_AP,year,pretty_domain
0,www.buzzfeednews.com/article/tasneemnashrulla/...,"""eat the babies"" viral video at aoc town hall ...",2019-10-04 00:00:00,buzzfeed,pro,cc,None,2019.0,Buzzfeed
1,www.buzzfeednews.com/article/passantino/extrem...,"""extremely likely"" global warming is man-made,...",2013-09-27 00:00:00,buzzfeed,pro,cc,None,2013.0,Buzzfeed
2,shotofprevention.com/2010/11/03/history-makes-...,"""history"" makes headlines with launch of new w...",2020-03-13 14:32:02,https://shotofprevention/,pro,vax,False,2020.0,Shot of Prevention
3,www.buzzfeednews.com/article/andrewkaczynski/i...,"""it's global warming, stupid"" - buzzfeed news",2012-11-01 00:00:00,buzzfeed,pro,cc,None,2012.0,Buzzfeed
4,www.buzzfeednews.com/article/tasneemnashrulla/...,"""japan dropped an atomic bomb on america durin...",2014-02-24 00:00:00,buzzfeed,pro,cc,None,2014.0,Buzzfeed
5,www.buzzfeednews.com/article/llevin/opinion-im...,"""look at my record, child"": joe biden showed m...",2019-10-31 00:00:00,buzzfeed,pro,cc,None,2019.0,Buzzfeed
6,www.breitbart.com/politics/2019/06/14/orourke-...,"""president o'rourke will end oil and gas lease...",2019-06-14 00:00:00,breitbart,anti,cc,None,2019.0,Breitbart
7,www.buzzfeednews.com/article/andrewkaczynski/s...,smoking doesnt kill and other great old opeds ...,2015-03-31 00:00:00,buzzfeed,pro,cc,None,2015.0,Buzzfeed
8,www.foxnews.com/world/100-carbon-tax-by-2030-c...,"$100 carbon tax by 2030 could save climate, sa...",2017-05-29 00:00:00,fox,anti,cc,None,2017.0,Fox
21585,childrenshealthdefense.org/news/4-billion-and-...,$4 billion and growing: u.s. payouts for vacc...,2018-11-19 00:00:00,chd,anti,vax,None,2018.0,Children's Health Defense


# Contents:
* [Define functions](#Define-functions)
* [Extract clausal complements with SpaCy](#Extract-embedded-clauses-with-SpaCy)

# Define functions

We add a neural coref resolution step to the default SpaCy pipeline: https://github.com/huggingface/neuralcoref

In [9]:
import spacy
nlp = spacy.load('en_core_web_sm')

# First way we can control a parameter
#neuralcoref.add_to_pipe(nlp, greedyness=0.75)
import neuralcoref
neuralcoref.add_to_pipe(nlp)

Helper functions for the main ```get_quotes``` function:

In [4]:
def is_good_verb_dep(dep):
    return dep[:3] == 'aux' or dep[:3] == 'adv' or dep == 'det' or dep == 'rel' or dep == 'prep' or dep[-3:] == 'obj' or dep[-3:] == 'mod' or dep == 'prt' or (dep[-4:] == 'comp' and dep != 'ccomp')

def is_good_subj_dep(dep):
    return dep != 'ccomp'

def is_ROOT(tok):
    return tok.dep_ == 'ROOT' or tok.dep_[-2:] == 'cl' or tok.dep_ == 'ccomp' or \
            (tok.dep_ == 'conj' and tok.head.dep_ == 'ROOT')

REL_PRONOUNS = set(['who', 'whom', 'whose', 'which', 'that'])
def is_rel_pronoun(tok):
    tok = tok.lower().strip()
    return tok in REL_PRONOUNS

In [46]:
def get_quotes(text):
    # Do coref resolution
    doc = nlp(text)
    text = doc._.coref_resolved
    
    quote_objs = []
    
    for sent in sent_tokenize(text):#d.sents:
        #print(sent)
        sent = nlp(sent)
        # Go through entire sentence, looking for verbs embedding a complement clause
        VERBS = np.unique([token.head for token in sent if token.dep_ == 'ccomp'])
        #print(VERBS)
       
        # Extract everything else for each VERB
        for VERB in VERBS:
            #print("\nCcomp dependency found! For quoting verb '{}'".format(VERB))
            # Extract the rest of the quoting verb
            verb_deps = [x for x in VERB.children if is_good_verb_dep(x.dep_)]
            #print("\tFound children:",verb_deps)
            for x in verb_deps:
                new_children = [c for c in x.children if is_good_verb_dep(c.dep_)]
                #print("\tAdding children of {}:".format(x.text),new_children)
                verb_deps.extend(new_children)
                #print("\tUpdated verb deps:",verb_deps)
            
            # If verb's head is not itself (i.e., verb is not the ROOT), 
            # recursively trace back to ROOT, then add all children of ROOT
            ROOT = VERB
            while not is_ROOT(ROOT):
                ROOT = ROOT.head
                #print('\t\tCurrent root:',ROOT)
            if VERB is not ROOT:
                verb_deps.append(ROOT) 
                
            #print("\tAdding children of ROOT...")
            root_deps = [x for x in ROOT.children if is_good_verb_dep(x.dep_)]
            #print("\tFound ROOT deps:",root_deps)
            for x in root_deps:
                new_children = [c for c in x.children if is_good_verb_dep(c.dep_)]
                #print("\tAdding children of {}:".format(x.text),new_children)
                root_deps.extend(new_children)
                #print("\tUpdated ROOT deps:",root_deps)

            #print("\tAdding ROOT deps to verb deps...")
            verb_deps.extend([x for x in root_deps if x != VERB and x not in verb_deps])
            #print("\tUpdated verb deps:",verb_deps)
            
            NEG,IS_NEG,neg_children = None,None,None
            SUBJECT,subj_children = None,None
                
            #print("\nLooking for SUBJECT and NEGATION(s)...")
            for child in ROOT.children:
                if child.dep_[:5] == 'nsubj' or child.dep_ == 'expl':
                    SUBJECT = child
                    #print("\tFound SUBJECT:",SUBJECT)
                    if SUBJECT.head.dep_ == 'relcl' and is_rel_pronoun(SUBJECT.text): # we're dealing with the subject of a rel clause
                        #print("\tFound quote inside a relative clause. Finding antecedent subject...")
                        SUBJECT = SUBJECT.head.head
                        #print("\tTrue subject:",SUBJECT)
                    #print("Subject token '{}' is in a coref cluster:".format(SUBJECT),SUBJECT._.in_coref)
                
                if child.dep_[:3] == 'neg':
                    NEG = child
                    verb_deps.append(NEG)
                    neg_children = [c for c in NEG.children if c != VERB]
                    #print("n\tAdding new NEG children:",neg_children)
                    for x in neg_children:
                        new_children = [c for c in x.children]
                        #print("\tNew NEG grandchildren:",new_children)
                        neg_children.extend(new_children)
                        #print("\tUpdated neg_children:",neg_children)
                    verb_deps.extend(neg_children)
                    #print("\tUpdated verb_deps:",verb_deps)
                    IS_NEG = VERB in NEG.head.children or VERB == NEG.head
            
            if SUBJECT is None and (ROOT.dep_ == 'acl' or ROOT.dep_ == 'advcl'):
                main_verb = ROOT.head
                #print([(c.text,c.dep_) for c in main_verb.children])
                SUBJS = [c for c in main_verb.children if c.dep_[:5] == 'nsubj' or c.dep_ == 'expl']
                SUBJECT = SUBJS[0] if len(SUBJS) > 0 else None
           
            # Get rest of subject tokens
            if SUBJECT is not None:
                #print("\nFound SUBJECT:",SUBJECT)
                #print("\tAdding children of SUBJECT...")
                subj_children = [c for c in SUBJECT.children if is_good_subj_dep(c.dep_)]
                #print("\tFound children:",subj_children)
                for x in subj_children:
                    new_children = [c for c in x.children if is_good_subj_dep(c.dep_)]
                    #print("\tAdding children of child {}:".format(x.text),new_children)
                    subj_children.extend(new_children)
                    #print("\tUpdated subject children:",subj_children)

            sorted_verb_tokens = sorted([(c,c.i) for c in verb_deps+[VERB]],key=lambda x:x[1])
            #print("\n\tSorted verb tokens:",sorted_verb_tokens)
            if SUBJECT is not None:
                sorted_subj_tokens = sorted([(c,c.i) for c in subj_children+[SUBJECT]],key=lambda x:x[1])
                #print("\tSorted subject tokens:",sorted_subj_tokens)
            else:
                sorted_subj_tokens = None
            if NEG is not None:
                sorted_neg_tokens = sorted([(c,c.i) for c in neg_children+[NEG]],key=lambda x:x[1])
            else:
                sorted_neg_tokens = None

            #print("\nFinding quote introduced by '{}'...".format(VERB))
            emb_main_verbs = [c for c in VERB.children if c.dep_ == 'ccomp']
            #print("\tMain verbs of embedded clause:",emb_main_verbs)
            #assert len(emb_main_verbs) <= 2
            for emb_main_verb in emb_main_verbs:
                #print("\tMain *verb*:",emb_main_verb)

                # Recursively get all children of main verb of embedded clause 
                #print("\nRecursively getting children of main verb of embedded clause...\n")
                #print("*"*50)
                children_queue = [x for x in emb_main_verb.children]
                #print("\tAdding children of main verb:",children_queue)
                for x in children_queue:
                    new_children = [c for c in x.children]
                    #print("\tAdding children of {}:".format(x.text),new_children)
                    children_queue.extend(new_children)
                    #print("\tNew children queue:",children_queue)

                # Sort children and matrix verb to be in correct order
                #print("\nSorting tokens in quote...")
                children_and_indices = [(c,c.i) for c in children_queue+[emb_main_verb]]
                sorted_ = sorted(children_and_indices,key=lambda x:x[1])
                quote_objs.append({'quote':[tup[0].text for tup in sorted_],
                        'verb tokens':[tup[0].text for tup in sorted_verb_tokens],
                        'main verb':VERB.text,
                       'subject tokens':[tup[0].text for tup in sorted_subj_tokens] if sorted_subj_tokens is not None else None,
                       'main subject':SUBJECT.text if SUBJECT is not None else None,
                       'neg tokens':[tup[0].text for tup in sorted_neg_tokens] if sorted_neg_tokens is not None else None,
                       'main neg':NEG.text if NEG is not None else None,
                       'is neg':IS_NEG})
    
    return quote_objs

# Extract clausal complements with SpaCy

(Did this part on the cluster.)

In [42]:
s = time.time()

In [43]:
time.time()-s

6.651726722717285

## First 100 URLs

In [47]:
for url_ix in range(0,1):
    start_time = time.time()
    curr_url = df.url.values[url_ix]
    quotes = get_quotes(get_fulltext(curr_url)[0])
    fname = get_fname(curr_url)
    with open('./extracted_quotes/{}.jsonlist'.format(fname),'w+') as f:
        for res in quotes:
            json.dump(res, f)
            f.write('\n')
    print('{}\tElapsed time in seconds:'.format(fname),(time.time()-start_time)/60.)

www.buzzfeednews.com[SEP]article[SEP]tasneemnashrulla[SEP]eat-babies-aoc-town-hall-pro-trump-troll-larouche	Elapsed time in seconds: 0.05031239986419678


In [32]:
curr_url_ix = 1
curr_url = df.url.values[curr_url_ix]
quotes = get_quotes(get_fulltext(curr_url)[0])

In [20]:
fname = get_fname(curr_url)
print(fname)
with open('./extracted_quotes/{}.jsonlist'.format(fname),'w+') as f:
    for res in quotes:
        json.dump(res, f)
        f.write('\n')

www.buzzfeednews.com[SEP]article[SEP]tasneemnashrulla[SEP]eat-babies-aoc-town-hall-pro-trump-troll-larouche


In [21]:
with open('./extracted_quotes/{}.jsonlist'.format(fname),'r') as f:
    data = f.readlines()
    print(len(data))
    
read_quotes = [json.loads(q.strip()) for q in data]
print(read_quotes[0])

28
{'quote': ['climate', 'change', 'is', 'a', 'hoax'], 'verb tokens': ['believes'], 'main verb': 'believes', 'subject tokens': ['the', 'pro', '-', 'Trump', 'LaRouche', 'PAC', ',', 'which', 'believes', ','], 'main subject': 'PAC', 'neg tokens': None, 'main neg': None, 'is neg': None}


In [22]:
read_quotes[1]

{'quote': ['climate', 'change', 'is', 'a', 'hoax', 'planted', 'the', 'troll'],
 'verb tokens': ['believes'],
 'main verb': 'believes',
 'subject tokens': ['the',
  'pro',
  '-',
  'Trump',
  'LaRouche',
  'PAC',
  ',',
  'which',
  'believes'],
 'main subject': 'PAC',
 'neg tokens': None,
 'main neg': None,
 'is neg': None}

# Retrieval of extracted complement clauses from filename

In [48]:
DIR_WITH_NAMES = './extracted_quotes/'
DIR_WITH_NOS = './extracted_quotes_2/'
N_WITH_NAMES = 7433
assert len(os.listdir(DIR_WITH_NAMES)) == N_WITH_NAMES
FNAMES_WITH_NAMES = set(os.listdir(DIR_WITH_NAMES))

In [52]:
# Cool, we've extracted the comp clauses of all the first 7433 URLs. No re-indexing needed.
for ix_url in range(0,N_WITH_NAMES):
    url = df.url.values[ix_url]
    fname = '{}.jsonlist'.format(get_fname(url))
    assert fname in FNAMES_WITH_NAMES

In [68]:
def get_quotes_from_file(url_ix,df,missing):
    url = df.url.values[url_ix]
    if url_ix < 7433:
        fname = get_fname(url)
        dir_ = 'extracted_quotes'
    else:
        fname = 'url_no_'+str(url_ix)
        dir_ = 'extracted_quotes_2'
    
    try:
        with open('./{}/{}.jsonlist'.format(dir_,fname),'r') as f:
            data = f.readlines()
        quotes = []
        for x in data:
            res = json.loads(x.strip())
            res.update({'source':url})                
            quotes.append(res)
            
        return quotes
    except FileNotFoundError:
        print(fname)
        missing.append(fname)

In [69]:
len(os.listdir(DIR_WITH_NAMES))+len(os.listdir(DIR_WITH_NOS)),len(df)

(30930, 30930)

In [70]:
missing = []
all_quotes = [get_quotes_from_file(url_ix,df,missing) for url_ix in range(0,len(df))]
all_quotes = [item for sublist in all_quotes for item in sublist]

In [71]:
len(missing),len(all_quotes)

(0, 612995)

In [73]:
pickle.dump(all_quotes,open('all_quotes.pkl','wb'))

In [72]:
def get_quotes_with_lemma(lemma,quote_list):
    return [q for q in quote_list if lemmatizer.lemmatize(q['main verb'].lower()) == lemma]

def get_quotes_with_stem(stem_set,quote_list):
    return [q for q in quote_list if ps.stem(q['main verb'].lower()) in stem_set]

def get_quotes_without_stem(stem_set,quote_list):
    return [q for q in quote_list if ps.stem(q['main verb'].lower()) not in stem_set]

## Filter to true indirect statements

In [74]:
# Remove quotes with question verb

QUESTION_WORDS = set(['who','what','when','where','why','how','if','whose','whether','which','whom','whence'])
QUESTION_VERBS = set(['ask','wonder','figure','guess','inquire','interrogate','question'])

def has_indirect_question(quote_obj):
    quote = quote_obj['quote']
    first_word = quote[0].lower()
    return first_word in QUESTION_WORDS

def has_question_verb(quote_obj):
    verb = quote_obj['main verb'].lower()
    return verb in QUESTION_VERBS

In [75]:
all_quotes = [q for q in all_quotes if not has_question_verb(q)]
len(all_quotes)

609859

In [78]:
# Set aside quotes with 'be' as main verb

BE_STEMS = set(['is','wa','’s','are',"'s",'be','’m','am','are','’re',"'m","'re"])

quotes_without_be = get_quotes_without_stem(BE_STEMS,all_quotes)
len(quotes_without_be)

554136

In [79]:
# Remove quotes with 'point' but not 'point_out' as main verb

point_quotes = get_quotes_with_stem(['point'],quotes_without_be)
point_out_quotes = [x for x in point_quotes if 'out' in x['verb tokens']]

true_point_out_quotes = []
false_point_out_quotes = []
for x in point_out_quotes:
    indices_out = [i for i,t in enumerate(x['verb tokens']) if t.lower() == 'out']
    prev_toks = [ps.stem(x['verb tokens'][i-1].lower()) for i in indices_out]
    if 'point' in prev_toks:
        true_point_out_quotes.append(x)
    else:
        false_point_out_quotes.append(x)
        
len(true_point_out_quotes),len(false_point_out_quotes)

(2051, 4)

In [80]:
false_point_out_quotes

[{'quote': ['I',
   'am',
   'going',
   'to',
   'ask',
   'that',
   'you',
   'look',
   'at',
   'him'],
  'verb tokens': ['point', 'him', 'out'],
  'main verb': 'point',
  'subject tokens': None,
  'main subject': None,
  'neg tokens': None,
  'main neg': None,
  'is neg': None,
  'source': 'rssfeeds.usatoday.com/~/613545646/0/news-opinion~A-he-said-she-said-or-a-violent-rape-What-its-like-to-testify-against-your-rapist/'},
 {'quote': ['”',
   'Kappenberger',
   'said',
   'of',
   'the',
   'Environmental',
   'Protection',
   'Agency',
   '’s'],
  'verb tokens': ['has', 'pointed', 'this', 'information', 'out', 'to', 'me'],
  'main verb': 'pointed',
  'subject tokens': ['anyone'],
  'main subject': 'anyone',
  'neg tokens': None,
  'main neg': None,
  'is neg': None,
  'source': 'dailycaller.com/2015/11/18/undetectable-the-impact-a-un-treaty-will-have-on-global-warming/'},
 {'quote': ['signature', 'global', 'warming', 'rule'],
  'verb tokens': ['has', 'pointed', 'this', 'informat

In [81]:
all_quotes = [q for q in quotes_without_be if q not in false_point_out_quotes]
len(all_quotes)

554132

In [82]:
# Remove quotes with a set of non-attributive verbs introducing comp clause
OTHER_VERBS = set(['make','made','cause','caus','ensur','like','love',
                    'help','mean','get','got','let','allow','want','becam',
                    'becom','go','goe','lead'])

non_other_verb_quotes = get_quotes_without_stem(OTHER_VERBS,all_quotes)
len(non_other_verb_quotes)

477533

In [83]:
# Remove quotes with 'seem', 'appear' as main verb

non_seem_quotes = get_quotes_without_stem(['seem','appear'],non_other_verb_quotes)
len(non_seem_quotes)#+len(seem_quotes),len(all_quotes)

475345

In [84]:
# Remove verb with non-alpha main verb

non_punc_main_verb_quotes = [q for q in non_seem_quotes if q['main verb'][0].lower().isalpha()]
len(non_punc_main_verb_quotes)

471279

In [85]:
non_punc_main_verb_quotes[32432]

{'quote': ['Proof',
  'of',
  'the',
  'experiment',
  "'s",
  'success',
  'is',
  'that',
  'the',
  'customer',
  'will',
  'pay',
  'for',
  'the',
  'technology',
  'and',
  'start',
  'using',
  'the',
  'technology'],
 'verb tokens': ['said'],
 'main verb': 'said',
 'subject tokens': ['Rossi'],
 'main subject': 'Rossi',
 'neg tokens': None,
 'main neg': None,
 'is neg': None,
 'source': 'www.foxnews.com/science/cold-fusion-experiment-major-success-or-complex-hoax'}

In [86]:
pickle.dump(non_punc_main_verb_quotes,open('filtered_quotes.pkl','wb'))

## Examine distribution of indirect statements

In [ ]:
# most common main verbs, subjects, amount of negation, sources

In [87]:
quotes = pickle.load(open('filtered_quotes.pkl','rb'))
len(quotes)

471279

In [89]:
all_main_verb_stems = [ps.stem(x['main verb']).lower() for x in quotes]
counted_main_verb_stems = Counter(all_main_verb_stems)
sorted(counted_main_verb_stems.items(),key=lambda x:x[1],reverse=True)

[('said', 90224),
 ('say', 45485),
 ('think', 18085),
 ('told', 12025),
 ('know', 11974),
 ('believ', 10025),
 ('show', 9118),
 ('found', 8970),
 ('suggest', 8126),
 ('see', 7116),
 ('note', 7020),
 ('argu', 6745),
 ('report', 5441),
 ('ad', 5076),
 ('have', 5036),
 ('claim', 4496),
 ('explain', 4369),
 ('tell', 4357),
 ('wrote', 4301),
 ('find', 3671),
 ('warn', 3320),
 ('ask', 3303),
 ('announc', 3247),
 ('hope', 3192),
 ('sure', 3151),
 ('estim', 3124),
 ('conclud', 2965),
 ('understand', 2665),
 ('expect', 2664),
 ('thought', 2657),
 ('consid', 2344),
 ('agre', 2320),
 ('point', 2239),
 ('predict', 2175),
 ('do', 2132),
 ('state', 2088),
 ('indic', 2020),
 ('declar', 1983),
 ('write', 1920),
 ('assum', 1880),
 ('acknowledg', 1877),
 ('had', 1868),
 ('realiz', 1823),
 ('determin', 1790),
 ('insist', 1751),
 ('decid', 1554),
 ('knew', 1547),
 ('need', 1468),
 ('admit', 1445),
 ('matter', 1421),
 ('reveal', 1365),
 ('feel', 1269),
 ('learn', 1206),
 ('worri', 1175),
 ('rememb', 1171),

A lot of quotes don't have a subject:

In [91]:
len([q for q in quotes if q['main subject'] is None])

50851

The most common subject stems--it looks like coref didn't fully do its job.

In [92]:
all_main_subj_stems = [ps.stem(x['main subject']).lower() for x in quotes if x['main subject'] is not None]
counted_main_subj_stems = Counter(all_main_subj_stems)
sorted(counted_main_subj_stems.items(),key=lambda x:x[1],reverse=True)

[('i', 29372),
 ('we', 17009),
 ('you', 11546),
 ('it', 9844),
 ('scientist', 6839),
 ('he', 5976),
 ('studi', 5839),
 ('research', 5284),
 ('report', 5077),
 ('peopl', 4661),
 ('trump', 4086),
 ('offici', 3646),
 ('’s', 3411),
 ('she', 2919),
 ('expert', 2831),
 ('they', 2576),
 ('one', 2195),
 ('obama', 2133),
 ('that', 2047),
 ('author', 1899),
 ('group', 1898),
 ('percent', 1678),
 ('agenc', 1535),
 ('thi', 1472),
 ('some', 1459),
 ('compani', 1455),
 ('sander', 1344),
 ('administr', 1283),
 ('govern', 1253),
 ('“', 1081),
 ('democrat', 1009),
 ('team', 999),
 ('critic', 971),
 ('us', 965),
 ('presid', 957),
 ('poll', 955),
 ('mani', 940),
 ('those', 936),
 ('state', 918),
 ('pruitt', 915),
 ('there', 914),
 ('data', 904),
 ('depart', 875),
 ('leader', 870),
 ('biden', 862),
 ('analysi', 849),
 ('other', 827),
 ('epa', 822),
 ('activist', 819),
 ('countri', 808),
 ('hi', 796),
 ('court', 783),
 ('republican', 782),
 ('organ', 777),
 ('brown', 775),
 ('environmentalist', 771),
 ('ti

Somewhere between 9.9k to 10.5k quotes have a negated main verb.

In [93]:
len([q for q in quotes if q['is neg']])

9952

In [95]:
len([q for q in quotes if q['main neg'] is not None])

10510

In [101]:
all_quote_domains = [df.loc[df.url==q['source']]['pretty_domain'].values[0] for q in quotes]
counted_quote_domains = Counter(all_quote_domains)
sorted(counted_quote_domains.items(),key=lambda x:x[1],reverse=True)

[('New York Times', 122967),
 ('Mother Jones', 47599),
 ('Fox', 37265),
 ('Washington Post', 30950),
 ('Christian Science Monitor', 30707),
 ('The Nation', 25223),
 ('Breitbart', 24479),
 ('Guardian (US)', 24388),
 ('Vox', 20552),
 ('Buzzfeed', 9909),
 ('Grist', 9125),
 ('Daily Caller', 8811),
 ("Children's Health Defense", 8303),
 ('Shot of Prevention', 7830),
 ('Alternet', 6459),
 ('Blaze', 6307),
 ('National Review', 5534),
 ('PJ Media', 5423),
 ('Drudgereport', 3986),
 ('USA Today', 3382),
 ('Reason', 2558),
 ('Newsweek', 2411),
 ('Quartz', 2379),
 ('The Progressive', 2086),
 ('Voices for Vaccines', 2035),
 ('The Verge', 1864),
 ('Democracy Now', 1618),
 ('American Thinker', 1568),
 ('CNS', 1447),
 ('NBC', 1413),
 ('New York Magazine', 1245),
 ('Redstate', 1196),
 ('Sgtreport', 1188),
 ('Vice', 1184),
 ('Activist Post', 1183),
 ('Infowars', 828),
 ('In These Times', 821),
 ('News With Views', 678),
 ('Citizens', 645),
 ('Hot Air', 557),
 ('The Week', 507),
 ('The American Spectator

# Additional pre-processing

## Add lemma, stem info

## Add NER tags?